<AccordionGroup>

<Accordion title="Basic syntax" defaultOpen="true">

In [1]:
# | echo: false
from vinyl.examples import cars

car = cars()

In [2]:
with car as c:
    c.select([c.Name, c.Horsepower], sort=c.Year)
    c.limit(10)

::: {.panel-tabset}

#### SQL


In [3]:
# | echo: false
print(c.to_sql(optimized=True))

SELECT
  "t0"."Year",
  "t0"."Name",
  "t0"."Horsepower"
FROM "cars" AS "t0"
ORDER BY
  "t0"."Year" ASC
LIMIT 10

#### Result

In [4]:
# | echo: false
print(c.execute("text"))

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Year                ┃ Name                      ┃ Horsepower ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ timestamp(6)        │ string                    │ float64    │
├─────────────────────┼───────────────────────────┼────────────┤
│ 1970-01-01 00:00:00 │ chevrolet chevelle malibu │      130.0 │
│ 1970-01-01 00:00:00 │ buick skylark 320         │      165.0 │
│ 1970-01-01 00:00:00 │ plymouth satellite        │      150.0 │
│ 1970-01-01 00:00:00 │ amc rebel sst             │      150.0 │
│ 1970-01-01 00:00:00 │ ford torino               │      140.0 │
│ 1970-01-01 00:00:00 │ ford galaxie 500          │      198.0 │
│ 1970-01-01 00:00:00 │ chevrolet impala          │      220.0 │
│ 1970-01-01 00:00:00 │ plymouth fury iii         │      215.0 │
│ 1970-01-01 00:00:00 │ pontiac catalina          │      225.0 │
│ 1970-01-01 00:00:00 │ amc ambassador dpl        │      190.0 │
└─────────────────────┴───────────────────────────┴────────────┘

#### AST

In [5]:
# | echo: false
c

r0 := DatabaseTable: cars
  Name             string
  Miles_per_Gallon float64
  Cylinders        int64
  Displacement     float64
  Horsepower       float64
  Weight_in_lbs    int64
  Acceleration     float64
  Year             timestamp(6)
  Origin           string

r1 := Project[r0]
  Year:       r0.Year
  Name:       r0.Name
  Horsepower: r0.Horsepower

r2 := Sort[r1]
  asc r1.Year

Limit[r2, n=10]

#### AST graph


In [6]:
# | echo: false
c.visualize()

╭────────────────────────╮             
                                                   │ Limit                  │             
                                                   │    Year: timestamp(6)  │             
                                                   │    Name: string        │             
                                                   │    Horsepower: float64 │             
                                                   ╰────────────────────────╯             
                                                                ▲                         
                                                                │                         
                                                                │                         
                                                                │                         
                                                   ╭────────────────────────╮             
                                                   │ Sort                   │             
                                                   │    Year: timestamp(6)  │             
                                             ╭────▶│    Name: string        │             
                                             │     │    Horsepower: float64 │             
                                             │     ╰────────────────────────╯             
                                             │                  ▲                         
                                             │                  │                         
                                             │                  │                         
                                             │                  │                         
                                  ╭────────────────────╮        │                         
                                  │ SortKey            │        │                         
                                  │    :: timestamp(6) │        │                         
                                  ╰────────────────────╯        │                         
                                             ▲                  │                         
                                             │                  │                         
                                             │                  │                         
                                             │                  │                         
                                  ╭────────────────────╮        │                         
                                  │ Year: Field        │        │                         
                                  │    :: timestamp(6) │        │                         
                                  ╰────────────────────╯        │                         
                                             ▲                  │                         
                                             │                  │                         
                                             │                  │                         
                                             ╰──────────────────│                         
                                                   ╭────────────────────────╮             
                                                   │ Project                │             
                                                   │    Year: timestamp(6)  │             
           ╭──────────────────────────────────────▶│    Name: string        │◀─╮          
           │                                       │    Horsepower: float64 │  │          
           │                                       ╰────────────────────────╯  │          
           │                                                    ▲              │          
           │                                           ╭────────│              │          
           │                                   

:::

</Accordion>

<Accordion title="Null handling">

In [7]:
# | echo: false
from vinyl.examples import birdstrikes

birdstrikes = birdstrikes()
from vinyl.constructors import if_else
from vinyl.examples import birdstrikes

birdstrikes = birdstrikes()

birdstrikes = birdstrikes.define_all(
    birdstrikes.Effect__Amount_of_damage,
    f=[lambda x: if_else(x == "None", None, x)],
)
birdstrikes = birdstrikes._create_table("birdstrikes")

In [8]:
from vinyl.constructors import coalesce

with birdstrikes as b:
    b.filter(b.Effect__Amount_of_damage == None)
    b.filter(b.Airport__Name != None)
    b.select({"airport_name": coalesce(b.Airport__Name, "Unknown")})

::: {.callout-warning}

`column is None` and `column is not None` will not work as expected because Python requires `is` statements to evaluate to True or False.

:::

::: {.panel-tabset}

#### SQL


In [9]:
# | echo: false
print(b.to_sql(optimized=True))

SELECT
  COALESCE("t0"."Airport__Name", 'Unknown') AS "airport_name"
FROM "birdstrikes " AS "t0"
WHERE
  "t0"."Effect__Amount_of_damage" IS NULL AND NOT "t0"."Airport__Name" IS NULL

#### Result

In [10]:
# | echo: false
print(b.execute("text"))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ airport_name                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                        │
├───────────────────────────────┤
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ WASHINGTON DULLES INTL ARPT   │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ BARKSDALE AIR FORCE BASE ARPT │
│ …                             │
└───────────────────────────────┘

#### AST

In [11]:
# | echo: false
b

r0 := DatabaseTable: birdstrikes 
  Airport__Name              string
  Aircraft__Make_Model       string
  Flight_Date                string
  Aircraft__Airline_Operator string
  Origin_State               string
  When__Phase_of_flight      string
  Wildlife__Size             string
  Wildlife__Species          string
  When__Time_of_day          string
  Cost__Other                int64
  Cost__Repair               int64
  Cost__Total_$              int64
  Speed_IAS_in_knots         float64
  Effect__Amount_of_damage   string

r1 := Filter[r0]
  IsNull(r0.Effect__Amount_of_damage)

r2 := Filter[r1]
  NotNull(r1.Airport__Name)

Project[r2]
  airport_name: Coalesce([r2.Airport__Name, 'Unknown'])

#### AST graph


In [12]:
# | echo: false
b.visualize()

╭─────────────────────────╮       
                                        │ Project                 │       
                                ╭──────▶│    airport_name: string │       
                                │       ╰─────────────────────────╯       
                                │                    ▲                    
                                │                    │                    
                                │                    │                    
                                │                    │                    
                        ╭──────────────╮             │                    
                        │ Coalesce     │             │                    
                        │    :: string │◀─╮          │                    
                        ╰──────────────╯  │          │                    
                                ▲         │          │                    
                                │         │          │                    
                                │         │          │                    
                                │         │          │                    
╭──────────────────────╮        │         ╭────────────────────╮          
│ Airport__Name: Field │        │         │ 'Unknown': Literal │          
│    :: string         │────────╯         │    :: string       │          
╰──────────────────────╯                  ╰────────────────────╯          
            ▲                                        │                    
            │                                        │                    
            │                                        │                    
            │                                        │                    
            │                                        │                    
            │                    ╭───────────────────────────────────────╮
            │                    │ Filter                                │
            │                    │    Airport__Name: string              │
            │                    │    Aircraft__Make_Model: string       │
            │                    │    Flight_Date: string                │
            │                    │    Aircraft__Airline_Operator: string │
            │                    │    Origin_State: string               │
            │                    │    When__Phase_of_flight: string      │
            ╰───────────────────▲│    Wildlife__Size: string             │
                                ││    Wildlife__Species: string          │
                                ││    When__Time_of_day: string          │
                                ││    Cost__Other: int64                 │
                                ││    Cost__Repair: int64                │
                                ││    Cost__Total_$: int64               │
                                ││    Speed_IAS_in_knots: float64        │
                                ││    Effect__Amount_of_damage: string   │
                                │╰───────────────────────────────────────╯
                                │                    ▲                    
                                │                    │                    
                                │                    │                    
                        ╭───────────────╮            │                    
                        │ NotNull       │            │                    
                        │    :: boolean │            │                    
                        ╰───────────────╯            │                    
                                ▲                    │                    
                                │                    │                    
                                │                    │                    
                                │                    │                    
                    ╭──────────────────────╮         │           

:::

</Accordion>

<Accordion title="Functions">

In [13]:
# | echo: false
from vinyl.examples import iris

iris = iris()

In [14]:
def perimeter(x, y):
    return 2 * (x + y)


with iris as i:
    i.define({"petalPerimeter": perimeter(i.petalLength, i.petalWidth)})

::: {.panel-tabset}

#### SQL


In [15]:
# | echo: false
print(i.to_sql(optimized=True))

SELECT
  "t0"."sepalLength",
  "t0"."sepalWidth",
  "t0"."petalLength",
  "t0"."petalWidth",
  "t0"."species",
  (
    "t0"."petalLength" + "t0"."petalWidth"
  ) * CAST(2 AS TINYINT) AS "petalPerimeter"
FROM "iris" AS "t0"

#### Result

In [16]:
# | echo: false
print(i.execute("text"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ sepalLength ┃ sepalWidth ┃ petalLength ┃ petalWidth ┃ species ┃ petalPerimeter ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ float64     │ float64    │ float64     │ float64    │ string  │ float64        │
├─────────────┼────────────┼─────────────┼────────────┼─────────┼────────────────┤
│         5.1 │        3.5 │         1.4 │        0.2 │ setosa  │            3.2 │
│         4.9 │        3.0 │         1.4 │        0.2 │ setosa  │            3.2 │
│         4.7 │        3.2 │         1.3 │        0.2 │ setosa  │            3.0 │
│         4.6 │        3.1 │         1.5 │        0.2 │ setosa  │            3.4 │
│         5.0 │        3.6 │         1.4 │        0.2 │ setosa  │            3.2 │
│         5.4 │        3.9 │         1.7 │        0.4 │ setosa  │            4.2 │
│         4.6 │        3.4 │         1.4 │        0.3 │ setosa  │            3.4 │
│         5.0 │        3.4 │         1.5 │        0.2 │ setosa  │            3.4 │
│         4.4 │        2.9 │         1.4 │        0.2 │ setosa  │            3.2 │
│         4.9 │        3.1 │         1.5 │        0.1 │ setosa  │            3.2 │
│           … │          … │           … │          … │ …       │              … │
└─────────────┴────────────┴─────────────┴────────────┴─────────┴────────────────┘

#### AST

In [17]:
# | echo: false
i

r0 := DatabaseTable: iris
  sepalLength float64
  sepalWidth  float64
  petalLength float64
  petalWidth  float64
  species     string

Project[r0]
  sepalLength:    r0.sepalLength
  sepalWidth:     r0.sepalWidth
  petalLength:    r0.petalLength
  petalWidth:     r0.petalWidth
  species:        r0.species
  petalPerimeter: r0.petalLength + r0.petalWidth * 2

#### AST graph


In [18]:
from IPython.display import HTML

HTML("<style>.container { width:100% !important; }</style>")

In [19]:
# | echo: false
i.visualize()

╭────────────────────────────╮                                                                   
                                                                                                                           │ Project                    │                                                                   
                                                                                                                           │    sepalLength: float64    │                                                                   
                                                                                                                           │    sepalWidth: float64     │                                                                   
           ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────▶│    petalLength: float64    │◀───────────────────────────────────────────────────────╮          
           │                                                                                                               │    petalWidth: float64     │ │                                                      │          
           │                                                                                                               │    species: string         │ │                                                      │          
           │                                                                                                    ╭─────────▶│    petalPerimeter: float64 │ │                                                      │          
           │                                                                                                    │          ╰────────────────────────────╯ │                                                      │          
           │                                                                                                    │               ▲         ▲               │                                                      │          
           │                                      ╭───────────────────────────────────────────────────────────────────────────────────────│               │                                                      │          
           │                                      │                                                             │               │         │               │                                                      │          
           │                                      │                                                 ╭───────────╯               │         │               │                                                      │          
╭────────────────────╮                  ╭───────────────────╮                 ╭────────────────╮    │          ╭───────────────╮│         │               │                                                      │          
│ sepalLength: Field │                  │ sepalWidth: Field │                 │ species: Field │    │          │ Multiply      ││         │               │                                                      │          
│    :: float64      │                  │    :: float64     │                 │    :: string   │────╯          │    :: float64 │◀─────────│───────────────│───────────────────────────────────╮                  │          
╰────────────────────╯                  ╰───────────────────╯                 ╰────────────────╯               ╰───────────────╯          │               │                                   │                  │          
           ▲                                      ▲                                    ▲                               ▲                  │               │                                   │                  │          
           │                                      │                                    │                               │                  │      

:::

</Accordion>

<Accordion title="Window Automation">

In [20]:
# | echo: false
from vinyl.examples import co2_concentration

co2_concentration = co2_concentration()

co2_concentration = co2_concentration.define(
    {"Date": co2_concentration.Date.cast("date")}
)
co2_concentration = co2_concentration._create_table("co2_concentration")

In [21]:
with co2_concentration as c:
    c.define(
        {"CO2_normalized": c.CO2 - c.CO2.mean()},
        by=[c.Date.dt.extract("month")],
    )
    c.select([c.CO2_normalized], sort=c.Date)

::: {.panel-tabset}

#### SQL


In [22]:
# | echo: false
print(c.to_sql(optimized=True))

WITH t1 AS (
  SELECT
    "t0"."Date",
    "t0"."CO2" - AVG("t0"."CO2") OVER (PARTITION BY EXTRACT(month FROM "t0"."Date") ROWS BETWEEN UNBOUNDED 
PRECEDING AND UNBOUNDED FOLLOWING) AS "CO2_normalized"
  FROM "co2_concentration " AS "t0"
)
SELECT
  "t1"."Date",
  "t1"."CO2_normalized"
FROM t1 AS t1
ORDER BY
  "t1"."Date" ASC

#### Result

In [23]:
# | echo: false
print(c.execute("text"))

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Date       ┃ CO2_normalized ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ date       │ float64        │
├────────────┼────────────────┤
│ 1958-03-01 │     -38.557288 │
│ 1958-04-01 │     -38.167797 │
│ 1958-05-01 │     -38.147167 │
│ 1958-07-01 │     -37.738333 │
│ 1958-08-01 │     -36.673667 │
│ 1958-09-01 │     -36.724833 │
│ 1958-11-01 │     -37.977833 │
│ 1958-12-01 │     -37.934667 │
│ 1959-01-01 │     -37.171695 │
│ 1959-02-01 │     -37.620862 │
│ …          │              … │
└────────────┴────────────────┘

#### AST

In [24]:
# | echo: false
c

r0 := DatabaseTable: co2_concentration 
  CO2  float64
  Date date

r1 := Project[r0]
  ExtractMonth(Date): ExtractMonth(r0.Date)
  CO2:                r0.CO2
  Date:               r0.Date
  CO2_normalized:     r0.CO2 - WindowFunction(func=Mean(r0.CO2), frame=RowsWindowFrame(table=r0, group_by=[ExtractMonth(r0.Date)]))

r2 := Project[r1]
  Date:           r1.Date
  CO2_normalized: r1.CO2_normalized

Sort[r2]
  asc r2.Date

#### AST graph


In [25]:
# | echo: false
c.visualize()

╭────────────────────────────╮          
                                                                         │ Sort                       │          
                                                                         │    Date: date              │          
                                                                         │    CO2_normalized: float64 │          
                                                                         ╰────────────────────────────╯          
                                                                         ▲              ▲                        
                                                                         │              │                        
                                                                         │              │                        
                                                                  ╭────────────╮        │                        
                                                                  │ SortKey    │        │                        
                                                                  │    :: date │        │                        
                                                                  ╰────────────╯        │                        
                                                                         ▲              │                        
                                                                         │              │                        
                                                                         │              │                        
                                                                         │              │                        
                                                                  ╭─────────────╮       │                        
                                                                  │ Date: Field │       │                        
                                                                  │    :: date  │       │                        
                                                                  ╰─────────────╯       │                        
                                                                         ▲              │                        
                                                                         │              │                        
                                                                         ╰──────╮       │                        
                                                                                │       │                        
                                                                                ╰───────│                        
                                                                         ╭────────────────────────────╮          
                                                                         │ Project                    │          
                                                   ╭────────────────────▶│    Date: date              │          
                                                   │                     │    CO2_normalized: float64 │          
                                                   │                     ╰────────────────────────────╯          
                                                   │                                   ▲▲                        
                                                   │                                   ││                        
                                                   │                                   ││                        
                                            ╭─────────────╮                ╭───────────────────────╮             
                                            │ Date: Field │                │ CO2_normalized: Field │             
                                            │    :: date  │                │    :: 

:::

</Accordion>

</AccordionGroup>